In [11]:
from __future__ import annotations

from typing import Tuple, Dict, List, Optional, Union


# Our libraries
import benchmarks # possibile_benchmarks -> {bm4, bm8, bm12, bm15}
import algorithms
import floydwarshall
from algorithms.utils import Race

# Python 3 standard libraries
import random
import copy
import matplotlib.pyplot as plt
import json






def make_picking_list (distances : Dict[int,Dict[int,int]], length : int) -> List[int]:
    """
    Given the distance matrix concerning the whole warehouse and the length of the picking 
    list, this method returns a picking list of randomly selected locations.
    Location 0 is never selected because it is considered to be the I/O point.
    
    :param distances: The distance matrix for the whole warehouse.
    :param length: The length of the picking list.
    :return: The picking list.
    
    """
    candidates = list(range(1, len(distances)))
    picking_list = []
    while len(picking_list) < length:
        location = candidates.pop (random.randint(0, len(candidates) - 1))
        picking_list.append (location)
    return picking_list




def clean_distances (picking_list : List[int], matrix : Dict[int,Dict[int,int]]) -> Dict[int,Dict[int,int]]:
    """
    Given a picking list and a matrix of distances, this method filters the matrix of
    distances keeping only the nodes visited in this specific picking list.
    
    :param picking_list: The picking list.
    :param matrix: The distance matrix.
    :return: A new distance matrix.
    
    """
    m : Dict[int,Dict[int,int]] = {}
    pkl = [0] + list(picking_list)
    
    for node in pkl:
        m[node] = {}
        dic = matrix.get(node)
        
        for node2 in pkl:
            m[node][node2] = dic.get(node2)

    return m



def clean_connections (picking_list : List[int], matrix : Dict[int,Dict[int,Set[int]]]) -> Dict[int,Dict[int,Set[int]]]:
    """
    Given a picking list and a matrix of connections, this method filters the matrix of
    connections keeping only the nodes visited in this specific picking list.
    
    :param picking_list: The picking list.
    :param matrix: The connections matrix.
    :return: A new connections matrix.
    
    """
    m : Dict[int,Dict[int,Set[int]]] = {}
    pkl = [0] + list(picking_list)
    
    for node in pkl:
        m[node] = {}
        dic = matrix.get(node)
        
        for node2 in pkl:
            m[node][node2] = set()
            
            for i in matrix[node][node2]:
                if i in pkl:
                    m[node][node2].add (i)
    
    return m






def test (distances, connections, picking_lists, iterations = 5):
    """
    :param distances: The list of distance matrices on which the algorithms must be tested - type(List[List[int]])
    :param it: The number of iterations per each algorithm - type(int)
    :param print_time: If TRUE the computational time is also registered
    
    """
    output = {}
    
    for index, (distance_matrix, connections_matrix, picking_list) in enumerate(zip(distances, connections, picking_lists)):

        
        output[index] = {
                        "picking_list" : list(picking_list),
                        }
        
        
        
        pso = algorithms.Mattia_PSO (distances=distance_matrix,
                             picking_list=picking_list,
                             paths=connections_matrix,
                             era = 10_000,
                             particles = 30,
                             max_noimp = 900,
                             print_every = 100,
                             particle_data = {
                                              "greediness" : 0.1,
                                              "beta" : 0.7,
                                              "check_paths" : 0.15,
                                              "deepsearch" : 0.1,
                                              "fulldeepsearch" : 0.5,
                                              "max_depth" : 2500,
                                              }
                    )

                  
         
        aco = algorithms.AntColony (distances = distance_matrix,
                                    picking_list=picking_list,
                                    pher_init=0.1, 
                                    ro=0.9, 
                                    Q=2.0, 
                                    alpha=1.0,
                                    beta=5.0,
                                    evaporate=False, 
                                    max_iter=10_000,
                                    max_noimp=900,
                                    print_every = 100
                                   )
        zhong = algorithms.Zhong_PSO (distances = distance_matrix,
                                      picking_list=picking_list,
                                      particles = 30,
                                      w = 0.8,
                                      lt = 1000,
                                      max_iter = 10_000,
                                      max_noimp = 900,
                                      print_every = 100
                                     )
        
        zhou = algorithms.Zhou_PSO (distances = distance_matrix,
                                    picking_list=picking_list,
                                    particles = 20,
                                    alpha = 0.4,
                                    beta = 0.1,
                                    gamma = 1.0,
                                    max_iter = 10_000,
                                    max_noimp = 900,
                                    print_every = 100,
                                    new_version = True
                                   )
        
        
        race = Race([pso, aco, zhong, zhou])
        race(n=iterations)
        output[index]["results"] = race.results
        
        
    return output

-------------------

### Problems generation 

<h6> Warehouse 1 </h6>

In [2]:
warehouse1 = floydwarshall.Warehouse (blocks = 1,
                                   racks_per_block = 30,
                                   locations_per_side = 11,
                                   locations_size = (2,2),
                                   aisles_size = 2,
                                   crossaisles_size = 4
                                  )

In [3]:
distances1, paths1 = floydwarshall.floyd_warshall (warehouse1.graph)

<h6> Warehouse 2 </h6>

In [4]:
warehouse2 = floydwarshall.Warehouse (blocks = 3,
                                   racks_per_block = 10,
                                   locations_per_side = 11,
                                   locations_size = (2,2),
                                   aisles_size = 2,
                                   crossaisles_size = 4
                                  )

In [5]:
distances2, paths2 = floydwarshall.floyd_warshall (warehouse2.graph)

<h6> Warehouse 3 </h6>

In [6]:
warehouse3 = floydwarshall.Warehouse (blocks = 3,
                                   racks_per_block = 30,
                                   locations_per_side = 11,
                                   locations_size = (2,2),
                                   aisles_size = 2,
                                   crossaisles_size = 4
                                  )

In [7]:
# ! Long computational time
distances3, paths3 = floydwarshall.floyd_warshall (warehouse3.graph) 

-----

### Case Study

<h6> Warehouse 1 - Picking list of 20 items </h6>

In [12]:
pl = []; dists = []; paths = []
for _ in range (10):
    pkl = make_picking_list(distances1, 20) 
    pl.append (pkl)
    dists.append (clean_distances(pkl, distances1))
    paths.append (clean_connections(pkl, paths1))

    
output0 = test (dists, paths, pl, 5)

Mattia_PSO [1/5]Mattia_PSO [2/5]
AntColony [2/5]
Mattia_PSO [3/5]Mattia_PSO [4/5]

AntColony [3/5]

Mattia_PSO [5/5]
AntColony [1/5]
AntColony [4/5]
AntColony [5/5]
Zhong_PSO [1/5]
Zhong_PSO [2/5]
Zhong_PSO [3/5]
Zhong_PSO [4/5]
Zhong_PSO [5/5]
Zhou_PSO [1/5]
Zhou_PSO [2/5]
Zhou_PSO [3/5]
Zhou_PSO [4/5]
Zhou_PSO [5/5]
Mattia_PSO [1/5]Mattia_PSO [2/5]AntColony [2/5]Mattia_PSO [3/5]Mattia_PSO [4/5]


AntColony [3/5]


Mattia_PSO [5/5]
AntColony [1/5]
AntColony [4/5]
AntColony [5/5]
Zhong_PSO [1/5]Zhong_PSO [2/5]

Zhong_PSO [3/5]
Zhong_PSO [4/5]
Zhong_PSO [5/5]
Zhou_PSO [1/5]
Zhou_PSO [2/5]
Zhou_PSO [3/5]
Zhou_PSO [4/5]
Zhou_PSO [5/5]
AntColony [2/5]Mattia_PSO [2/5]Mattia_PSO [3/5]Mattia_PSO [4/5]Mattia_PSO [1/5]



AntColony [3/5]

AntColony [1/5]
Mattia_PSO [5/5]
AntColony [4/5]
AntColony [5/5]
Zhong_PSO [1/5]
Zhong_PSO [2/5]
Zhong_PSO [3/5]
Zhong_PSO [4/5]
Zhong_PSO [5/5]
Zhou_PSO [1/5]
Zhou_PSO [2/5]
Zhou_PSO [3/5]
Zhou_PSO [4/5]
Zhou_PSO [5/5]
Mattia_PSO [1/5]AntColony [2/5]Mattia_PS

-----

<h6> Warehouse 1 - Picking list of 30 items </h6>

In [ ]:
pl = []; dists = []; paths = []
for _ in range (10):
    pkl = make_picking_list(distances1, 30) 
    pl.append (pkl)
    dists.append (clean_distances(pkl, distances1))
    paths.append (clean_connections(pkl, paths1))
    
output1 = test (dists, paths, pl, 5)

AntColony [1/5]Mattia_PSO [4/5]AntColony [2/5]Mattia_PSO [3/5]Mattia_PSO [5/5]AntColony [3/5]





Mattia_PSO [1/5]
Mattia_PSO [2/5]
AntColony [4/5]
AntColony [5/5]
Zhong_PSO [1/5]
Zhong_PSO [2/5]
Zhong_PSO [3/5]
Zhong_PSO [4/5]
Zhong_PSO [5/5]
Zhou_PSO [1/5]
Zhou_PSO [2/5]
Zhou_PSO [3/5]
Zhou_PSO [4/5]
Zhou_PSO [5/5]
Mattia_PSO [3/5]AntColony [1/5]Mattia_PSO [4/5]
Mattia_PSO [5/5]AntColony [2/5]
AntColony [3/5]



Mattia_PSO [1/5]Mattia_PSO [2/5]

AntColony [5/5]AntColony [4/5]

Zhong_PSO [1/5]
Zhong_PSO [2/5]
Zhong_PSO [3/5]
Zhong_PSO [4/5]
Zhong_PSO [5/5]
Zhou_PSO [1/5]
Zhou_PSO [2/5]
Zhou_PSO [3/5]
Zhou_PSO [4/5]
Zhou_PSO [5/5]
AntColony [1/5]Mattia_PSO [3/5]AntColony [2/5]Mattia_PSO [4/5]Mattia_PSO [5/5]


AntColony [3/5]


Mattia_PSO [2/5]Mattia_PSO [1/5]

AntColony [4/5]
AntColony [5/5]
Zhong_PSO [1/5]
Zhong_PSO [2/5]
Zhong_PSO [3/5]
Zhong_PSO [4/5]
Zhong_PSO [5/5]
Zhou_PSO [1/5]
Zhou_PSO [2/5]
Zhou_PSO [3/5]
Zhou_PSO [4/5]
Zhou_PSO [5/5]
AntColony [1/5]Mattia_PSO [3/5]AntColony

----

<h6> Warehouse 1 - Picking list of 40 items </h6>

In [17]:
pl = []; dists = []; paths = []
for _ in range (10):
    pkl = make_picking_list(distances1, 40) 
    pl.append (pkl)
    dists.append (clean_distances(pkl, distances1))
    paths.append (clean_connections(pkl, paths1))
    
output2 = test (dists, paths, pl, 5)

AntColony [1/5]Mattia_PSO [3/5]AntColony [2/5]Mattia_PSO [4/5]
Mattia_PSO [5/5]


AntColony [3/5]

Mattia_PSO [2/5]
Mattia_PSO [1/5]
AntColony [4/5]
AntColony [5/5]
Zhong_PSO [1/5]
Zhong_PSO [2/5]
Zhong_PSO [3/5]
Zhong_PSO [4/5]
Zhong_PSO [5/5]
Zhou_PSO [1/5]
Zhou_PSO [2/5]
Zhou_PSO [3/5]
Zhou_PSO [4/5]
Zhou_PSO [5/5]
Mattia_PSO [3/5]
Mattia_PSO [4/5]AntColony [1/5]AntColony [2/5]

AntColony [3/5]Mattia_PSO [5/5]


Mattia_PSO [1/5]Mattia_PSO [2/5]

AntColony [4/5]
AntColony [5/5]
Zhong_PSO [1/5]
Zhong_PSO [2/5]
Zhong_PSO [3/5]
Zhong_PSO [4/5]
Zhong_PSO [5/5]
Zhou_PSO [1/5]
Zhou_PSO [2/5]
Zhou_PSO [3/5]
Zhou_PSO [4/5]
Zhou_PSO [5/5]
AntColony [1/5]Mattia_PSO [3/5]AntColony [2/5]AntColony [3/5]Mattia_PSO [4/5]


Mattia_PSO [5/5]


Mattia_PSO [1/5]Mattia_PSO [2/5]

AntColony [4/5]
AntColony [5/5]
Zhong_PSO [1/5]
Zhong_PSO [2/5]
Zhong_PSO [3/5]
Zhong_PSO [4/5]
Zhong_PSO [5/5]
Zhou_PSO [1/5]
Zhou_PSO [2/5]
Zhou_PSO [3/5]
Zhou_PSO [4/5]
Zhou_PSO [5/5]
Mattia_PSO [3/5]Mattia_PSO [4/5]AntColon

----

<h6> Warehouse 2 - Picking list of 20 items </h6>

In [18]:
pl = []; dists = []; paths = []
for _ in range (10):
    pkl = make_picking_list(distances2, 20) 
    pl.append (pkl)
    dists.append (clean_distances(pkl, distances2))
    paths.append (clean_connections(pkl, paths2))
    
output3 = test (dists, paths, pl, 5)

AntColony [2/5]AntColony [1/5]Mattia_PSO [3/5]Mattia_PSO [5/5]


Mattia_PSO [4/5]AntColony [3/5]


Mattia_PSO [2/5]
Mattia_PSO [1/5]
AntColony [4/5]
AntColony [5/5]
Zhong_PSO [1/5]
Zhong_PSO [2/5]
Zhong_PSO [3/5]
Zhong_PSO [4/5]
Zhong_PSO [5/5]
Zhou_PSO [1/5]
Zhou_PSO [2/5]
Zhou_PSO [3/5]
Zhou_PSO [4/5]
Zhou_PSO [5/5]
Mattia_PSO [3/5]AntColony [1/5]AntColony [2/5]Mattia_PSO [4/5]Mattia_PSO [5/5]



AntColony [3/5]

Mattia_PSO [1/5]
Mattia_PSO [2/5]
AntColony [4/5]
AntColony [5/5]
Zhong_PSO [1/5]
Zhong_PSO [2/5]
Zhong_PSO [3/5]
Zhong_PSO [4/5]
Zhong_PSO [5/5]
Zhou_PSO [1/5]
Zhou_PSO [2/5]
Zhou_PSO [3/5]
Zhou_PSO [4/5]
Zhou_PSO [5/5]
AntColony [1/5]AntColony [2/5]Mattia_PSO [3/5]Mattia_PSO [4/5]
Mattia_PSO [5/5]



AntColony [3/5]
Mattia_PSO [1/5]Mattia_PSO [2/5]

AntColony [4/5]
AntColony [5/5]
Zhong_PSO [1/5]
Zhong_PSO [2/5]
Zhong_PSO [3/5]
Zhong_PSO [4/5]
Zhong_PSO [5/5]
Zhou_PSO [1/5]
Zhou_PSO [2/5]
Zhou_PSO [3/5]
Zhou_PSO [4/5]
Zhou_PSO [5/5]
AntColony [1/5]AntColony [2/5]Mattia_PSO

-----

<h6> Warehouse 2 - Picking list of 30 items </h6>

In [19]:
pl = []; dists = []; paths = []
for _ in range (10):
    pkl = make_picking_list(distances2, 30) 
    pl.append (pkl)
    dists.append (clean_distances(pkl, distances2))
    paths.append (clean_connections(pkl, paths2))

output4 = test (dists, paths, pl, 5)

--------

<h6> Warehouse 2 - Picking list of 40 items </h6>

In [20]:
pl = []; dists = []; paths = []
for _ in range (10):
    pkl = make_picking_list(distances2, 40) 
    pl.append (pkl)
    dists.append (clean_distances(pkl, distances2))
    paths.append (clean_connections(pkl, paths2))

output5 = test (dists, paths, pl, 5)

--------

<h6> Warehouse 3 - Picking list of 20 items </h6>

In [21]:
pl = []; dists = []; paths = []
for _ in range (10):
    pkl = make_picking_list(distances3, 20) 
    pl.append (pkl)
    dists.append (clean_distances(pkl, distances3))
    paths.append (clean_connections(pkl, paths3))
    
output6 = test (dists, paths, pl, 5)

-----------------

<h6> Warehouse 3 - Picking list of 30 items </h6>

In [22]:
pl = []; dists = []; paths = []
for _ in range (10):
    pkl = make_picking_list(distances3, 30) 
    pl.append (pkl)
    dists.append (clean_distances(pkl, distances3))
    paths.append (clean_connections(pkl, paths3))
    
output7 = test (dists, paths, pl, 5)

------------------

<h6> Warehouse 3 - Picking list of 40 items </h6>

In [23]:
pl = []; dists = []; paths = []
for _ in range (10):
    pkl = make_picking_list(distances3, 40) 
    pl.append (pkl)
    dists.append (clean_distances(pkl, distances3))
    paths.append (clean_connections(pkl, paths3))
    
output8 = test (dists, paths, pl, 5)

---------

In [31]:
outputs = (output0, output1, output2, output3, output4, output5, output6, output7, output8)

In [55]:
with open("results.txt", "a") as file:
    print("[", end="")
    for config, o in enumerate(outputs):
        #file.write ("Configuration " + str(config) + "\n")
        #file.write("-------------------------------------------------------------------\n")
        
        for i, res in enumerate(o.values()):
            print (json.dumps(res["picking_list"]), ",")
    print("]")
            #for j, r in enumerate(res["results"]):
            #    if j == 0:
            #        file.write ("Cost;Computations;ComputationalTime;" * 5 + "\n")
            #        
            #    string = str(r["cost"]) + ";" + str(r["computations"]) + ";" + str(r["computational_time"]).replace(".",",") + ";"
            #    if j == 0 or r["alg"] == res["results"][j-1]["alg"]:
            #        file.write(string)
            #    else:
            #        file.write("\n")
            #        file.write(string)
            #file.write ("\n\n")


[[302, 309, 61, 356, 311, 210, 270, 378, 251, 372, 380, 78, 119, 242, 74, 135, 209, 55, 262, 166] ,
[3, 225, 56, 280, 156, 306, 309, 11, 169, 16, 168, 165, 238, 147, 80, 323, 123, 203, 104, 391] ,
[84, 367, 56, 355, 78, 39, 119, 301, 65, 172, 158, 359, 120, 400, 5, 71, 197, 102, 384, 196] ,
[237, 202, 115, 62, 363, 356, 267, 149, 192, 397, 158, 296, 360, 314, 57, 20, 304, 200, 161, 325] ,
[282, 310, 100, 17, 333, 205, 49, 139, 16, 243, 296, 135, 79, 52, 206, 129, 253, 331, 263, 191] ,
[388, 317, 357, 153, 350, 66, 148, 320, 53, 188, 401, 383, 326, 391, 13, 35, 183, 271, 344, 264] ,
[7, 313, 401, 303, 285, 245, 277, 351, 23, 230, 35, 262, 61, 372, 217, 101, 105, 209, 289, 164] ,
[121, 86, 234, 150, 383, 185, 123, 381, 119, 110, 111, 18, 224, 322, 120, 12, 357, 130, 235, 80] ,
[58, 39, 114, 17, 178, 253, 64, 326, 300, 383, 187, 359, 59, 20, 173, 274, 98, 204, 251, 8] ,
[7, 265, 230, 183, 177, 358, 121, 396, 165, 61, 330, 163, 9, 21, 34, 205, 189, 341, 110, 389] ,
[10, 299, 254, 362, 280,